# Introduction
The Nominatim API is a REST API (provided by OpenStreetMap), that returns geographic coordinates (latitude and longitude) from a postal address. If several postal addresses (due to inaccuracy or non-existent street number) corresponding to the request are found, the API returns several coordinates, each time with an `importance` score. The coordinates are given in descending order of importance, so you can select only the first address.

In [1]:
# Execute the code below :

import requests
link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0',
    'Referer': 'https://www.example.com'
}
r = requests.get(link, headers=headers).json()
r

[{'place_id': 46158390,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 1203329854,
  'lat': '37.7242361',
  'lon': '15.1789984',
  'class': 'highway',
  'type': 'residential',
  'place_rank': 26,
  'importance': 0.10000999999999993,
  'addresstype': 'road',
  'name': 'Via Pietro Mascagni',
  'display_name': 'Via Pietro Mascagni, Altarello, Giarre, Catania, Sicilia, 95014, Italia',
  'boundingbox': ['37.7234970', '37.7249937', '15.1784912', '15.1794786']},
 {'place_id': 46193870,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 128394486,
  'lat': '37.6090903',
  'lon': '15.1604414',
  'class': 'highway',
  'type': 'residential',
  'place_rank': 26,
  'importance': 0.10000999999999993,
  'addresstype': 'road',
  'name': 'Via Pietro Mascagni',
  'display_name': 'Via Pietro Mascagni, Santa Caterina, Acireale, Catania, Sicilia, 95024, Italia',
  'bou

In [2]:
# Here we select only the first address (index 0)
print("First address :", r[0])
print("First address longitude :",r[0]['lon'])
print("First address latitude :",r[0]['lat'])

First address : {'place_id': 46158390, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 1203329854, 'lat': '37.7242361', 'lon': '15.1789984', 'class': 'highway', 'type': 'residential', 'place_rank': 26, 'importance': 0.10000999999999993, 'addresstype': 'road', 'name': 'Via Pietro Mascagni', 'display_name': 'Via Pietro Mascagni, Altarello, Giarre, Catania, Sicilia, 95014, Italia', 'boundingbox': ['37.7234970', '37.7249937', '15.1784912', '15.1794786']}
First address longitude : 15.1789984
First address latitude : 37.7242361


In [3]:
# For ease of use, you can add a limit on the number of items returned:

link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json"
r = requests.get(link, headers=headers).json()
print("WITHOUT limit, how many coordinates does this address return?",len(r))

link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json&limit=1"
r = requests.get(link, headers=headers).json()
print("WITH limit, how many coordinates does this address return?",len(r))

WITHOUT limit, how many coordinates does this address return? 10
WITH limit, how many coordinates does this address return? 1


## How to create your API query
It's up to you to modify the string to create the right request URL

In [4]:
# We observe that the query consists of a fixed part, followed by the address to be searched for.
# An URL cannot contain a " " space character,
# and special characters or accents should be avoided if possible

link_main = 'https://nominatim.openstreetmap.org/?q='
address = '54 Via Pietro Mascagni, Catania, Italy'
link_end = '&format=json&limit=1'

link = link_main + address.replace(' ','+').replace(',','')# It's your turn here. Your goal is to get a link in the right format

print(link + link_end)

https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni+Catania+Italy&format=json&limit=1


In [5]:
# Create a function here that turns a postal address into a request URL for the Nominatim API,
# then makes the request and returns the coordinates :

def API_address(postal_address):
  link_main = 'https://nominatim.openstreetmap.org/?q='
  link = link_main + postal_address.replace(' ','+').replace(',','') + link_end
  r = requests.get(link, headers=headers).json()
  return (r[0]['lon'], r[0]['lat'])[::-1]


In [6]:
# Test it here:
API_address(address)

('37.7242361', '15.1789984')

# DataViz
Latitude & Longitude can be used on visualization tools, whether they are BI tools (PowerBI, Table), or Python DataViz libraries such as Plotly or Folium.

Here we will display a map with Folium.


In [7]:
# The syntax of Folium is very simple, you start by creating a map centered on a point.
# You can change the default zoom level with the argument "zoom_start".

import folium

# We define a location as a list with 2 values : latitude and longitude.
point = [float(r[0]['lat']), float(r[0]['lon'])]

# We center the map on the location
m = folium.Map(location=point,zoom_start=7)

# We display the map
m

In [8]:
# Then you can add landmarks and put a clickable comment
m = folium.Map(location=point, )
folium.Marker(
    location=point,
    popup='a good restaurant'
    ).add_to(m)
m

# Challenge
Here is a DataFrame with restaurants in Catania, Sicily, and their respective addresses. Here is your mission:
- Create a new column "coordinates", which will store the coordinates corresponding to each address (you can use the function you created previously).
- Display a map with the 4 restaurant markers. Be careful, the restaurants are very close, remember to set the default zoom level so that it is clearly legible. You can center the map on the first restaurant. And display the name of the restaurant in the tooltip popup.

In [9]:
import pandas as pd
restaurants = pd.DataFrame([["Gelateria Zio Pietro dal 1964", "Via Porta di Ferro, 47, 95131 Catania CT"],
                            ["A Casa d'Amici","Via Fischetti, 14, 95131 Catania CT"],
                            ["La Bitta", "Via Acquicella Porto, 95121 Catania CT"],
                            ["Steak House", "Via Porta di Ferro, 8, 95100 Catania CT"]
                            ],
                           columns = ["name", "address"])

restaurants

,name,address
0,Gelateria Zio Pietro dal 1964,"Via Porta di Ferro, 47, 95131 Catania CT"
1,A Casa d'Amici,"Via Fischetti, 14, 95131 Catania CT"
2,La Bitta,"Via Acquicella Porto, 95121 Catania CT"
3,Steak House,"Via Porta di Ferro, 8, 95100 Catania CT"


In [10]:
restaurants['coordinates'] = restaurants['address'].apply(API_address)

In [11]:
restaurants

,name,address,coordinates
0,Gelateria Zio Pietro dal 1964,"Via Porta di Ferro, 47, 95131 Catania CT","(37.5020931, 15.0931951)"
1,A Casa d'Amici,"Via Fischetti, 14, 95131 Catania CT","(37.506171, 15.0943223)"
2,La Bitta,"Via Acquicella Porto, 95121 Catania CT","(37.4889155, 15.0833748)"
3,Steak House,"Via Porta di Ferro, 8, 95100 Catania CT","(37.5029335, 15.0930406)"


In [12]:
for index, row in restaurants.iterrows():
  folium.Marker(
    location=row['coordinates'],
    popup = row['name']
    ).add_to(m)
m

# Remarks on the Nominatim API
As indicated in the quest, there are many resources for geocoding. Most of them are available by registration, and some require a fee.

The Nominatim API is free and without registration. The disadvantage is that it is relatively slow. If you need to use it in the future, remember to store the results so you don't have to run it several times.

For your knowledge, there is also :
- the **reverse** address API, which allows you to find the nearest postal address using geographic coordinates.
- the API from a **CSV file** if you have a lot of addresses to geocode
- the API **GeoJSON** which allows to obtain a geoJSON format of locations to make choropleth maps


All the [documentation is available here](https://nominatim.org/release-docs/develop/api/Search/).